In [1]:
# Run this line below into the console while in the directory of where the images will be saved.

# ffmpeg -start_number 0 -framerate 60 -i graph%01d.png video.webm

In this notebook we want to lookd a few different numerical wave phenomenon. I just think waves are cool and PDEs are some of my favorite structures in all of mathematics. That being said, I ultimately want to make some wave simulations under a few different constraints. My big goals are as follows:
- Create a basic 2D wave progpogation simulation
- Create a similar sort of equation for Schrodinger equation
- Create a 3D wave simulation

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from matplotlib import cm
from matplotlib.animation import FuncAnimation
import matplotlib as mpl
from tqdm import tqdm

In [3]:
#We have this cell here to call the dataAnalysisLibrary module from one place rather than copy pasting into many other places
# the ../ means one directory back, which is, so this should fix pathing problems moving forward

sys.path.insert(0,'../Code/')
import simulationLibrary as sim

In [6]:
sim.clearDirectory()

# Basic Wave Simulation

To remind ourselves, the basic wave equation is $\frac{\partial^2 u}{\partial t^2}=c^2\nabla^2 u$. This is the generalized wave equation for multiple dimensions, for a 2d eq we simply need to reduce it down to break down that laplacian operator and we will get the simple wave equation $\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$. Why the partials, you might ask? Well that is because out position function, $u$, is a function of two variables, namely position and time. That is, $u$ is secretly $u(x,t)$ under the hood. When $t=0$ then we $u$ becomes $u(x,0)$, or, more compactly, just $u(x)$. THIS is just a curve! A 2D curve, specifically. A cross section of the wave at a certain time, specifically at our starting time $t=0$. If we advance time to anything else, then the wave should also change (It doesnt have to change, but the purposes of modeling a physical system it probably should, so moving forward we should be expecting this sort of thing). So, thus $u(x,t)$ is a function of curves, varied by space and time, that contains all the information of the wave that we want to get. That is why we are dealing with partial derivatives here. And since we are dealing with partial derivatives, how can we actually turn this into a numerical scheme that we can computer on a computer?

This question takes us to the realm of numerical analysis. Numerical analysis' entire existence is based upon approximating smooth curves and other sort of numerical algorithms. What do I mean by "numerical algorithms" or just "numerical" in general? Well, as with all words, it depends on the context. In a mathematical sense, numerical means approximation. So if you hear the term "Numerical methods" you can replace it in your mind with "approximate methods". This field is required for things that have no analytical solution, that is something that cant be explicitly solved with pen an paper. Mostly everything in nonanalytical so numerical methods are everywhere. We will be using a few things from numerical methods in this. The first thing is to find out how to represent our wave equation in a way that our computer can process. We can do this using finite difference methods for partial derivatives. Ignoring the derivation of it and going off a "trust me" sort of thing, this is the finite difference approximation for a second order partial derivative. Let $u(x,t)$ be our mutlivariate function and lets approximate it at a specific point, that is at $x_0$ and $t_0$

$$
\frac{\partial^2 u}{\partial x^2}(x_0,t_0) \approx \frac{u(x_0+\Delta x,t_0)-2u(x_0,t_0)+f(x_0-\Delta x,t_0)}{(\Delta x)^2}.
$$

This is nice, but unfortunately the field of partial differential calculus loves to use many different forms of notation for the same thing. Because of this we are gonna have to use some other notation that im not entirely fond of due to its non-ituitiveness, but alas here we are! Anyways, the notation is this:

$$
u^{(m)}_j=\tilde{u}(x_j,t_m)
$$

Where $\tilde{u}$ is the approximation of $u$ at time $t_m$ and position $x_j$. You know what? Looking back at this notation, its not that bad. I still have some problems with it, but I may have been a little hasty in my analysis of it. The approximation in this case is the central finite difference approximation that was stated up above. Using this notation, lets refine our central finite difference notation and clean up our equations a little bit using this new notation.

*********NOW INTRODUCTING A NEW NOTATION*********

We are going to derive a difference-equation method to compute approximations $\{u_{i,j}:i=1,2,...,n\}$ in successive rows for $j=2,3,...,m$

Replacing the double derivatives in the original wave equation with these new approximations gives us

$$
\frac{u_{i,j+1}-2u_{i,j}+u_{i,j-1}}{k^2}=c^2\frac{u_{i+1,j}-2u_{i,j}+u_{i-1,j}}{h^2}
$$

And, for ease, lets say $r=\frac{ck}{h}$, then our equation simplifies down to

$$
u_{i,j+1}-2u_{i,j}+u_{i,j-1}=r^2(u_{i+1,j}-2u_{i,j}+u_{i-1,j})
$$

Now solve for $u_{i,j+1}$

$$
u_{i,j+1}=(2-2r^2)u_{i,j}+r^2(u_{i+1,j}+u_{i-1,j})-u_{i,j-1}
$$

We will start at the bottom row, where $t=t_1=0$ and the solution is known to be $u(x_i,t_1)=u(x_i,0)=f(x_i)$ (aka its the initial condition). So, the true solution value at the mesh points of our grid is $u(x_i,t_j)$

In [7]:
def func(x):
    return(x*((2*np.pi)-x))

In [8]:
def gaussianDistribution(x, mean, std_dev):
    coefficient = 1 / (std_dev * np.sqrt(2 * np.pi))
    exponent = -((x - mean) ** 2) / (2 * std_dev ** 2)
    return(coefficient * np.exp(exponent))

# Traditional Wave Equation

In [43]:
def waveSimulation(initialFunc, c=1, length=1, totalTime=5, Nx=200, Nt=1000):
    # Lets create a meshed function
    length = length
    c=c
    totalTime = totalTime
    Nx = Nx # x axis
    Nt = Nt # t axis
    deltax = length/(Nx-1) # THIS IS THE IMPORTANT LINE OR ELSE IT BLOWS UP!!!!!!!!!!!!!!!!!!!! I HAVE NO CLUE WHY THAT -1 NEEDS TO BE THERE BUT IT DOES OR EVERYTHING GOES TO SHIT
    deltat = totalTime/Nt
    C=c*(deltat/deltax)
    if C >=1 :
        print("C is greater than 1. C="+str(C))
        sys.exit(1)
    
    x=np.linspace(0,length,Nx)
    t=np.linspace(0,totalTime, Nt)
    
    u=np.zeros((Nt, Nx))
    u[0]=initialFunc(x)
    u[1]=u[0]
    for n in tqdm(range(1,Nt-1)):
        for i in range(1,Nx-1): # All interior points
            u[n+1,i]=(2 * (1 - C**2) * u[n, i] - u[n - 1, i] + C**2 * (u[n, i + 1] + u[n, i - 1]))

    sim.clearDirectory()
    for i in tqdm(range(Nt)):
        plt.plot(u[i])
        plt.xlim(0,Nx)
        plt.ylim(-5,5)
        plt.savefig('../..//Physics-Simulations/Images for simulation/graph'+str(i)+'.png') # dpi argument increases resolution
        plt.close('all')

In [49]:
sim.clearDirectory()

# Damped Wave Equation

In [42]:
def gaussianDistribution(x, mean=0.5, std_dev=0.1):
    coefficient = 1 / (std_dev * np.sqrt(2 * np.pi))
    exponent = -((x - mean) ** 2) / (2 * std_dev ** 2)
    return(coefficient * np.exp(exponent))

In [47]:
def dampedWaveSimulation(initialFunc, c=1, gamma=1, length=1, totalTime=5, Nx=200, Nt=1000):
    # Lets create a meshed function
    length = length
    c=c
    gamma = gamma
    totalTime = totalTime
    Nx = Nx # x axis
    Nt = Nt # t axis
    deltax = length/(Nx-1) # THIS IS THE IMPORTANT LINE OR ELSE IT BLOWS UP!!!!!!!!!!!!!!!!!!!! I HAVE NO CLUE WHY THAT -1 NEEDS TO BE THERE BUT IT DOES OR EVERYTHING GOES TO SHIT
    deltat = totalTime/Nt
    C=c*(deltat/deltax)
    if C >=1 :
        print("C is greater than 1. C="+str(C))
        sys.exit(1)
    
    x=np.linspace(0,length,Nx)
    t=np.linspace(0,totalTime, Nt)
    
    u_initial = np.exp(-((x - 0.1)**2) / (2 * sigma**2))
    v_initial = (A * v * (x - x_0) / sigma**2) * np.exp(-((x - x_0)**2) / (2 * sigma**2))
    
    u=np.zeros((Nt, Nx))
    u[0]=initialFunc(x)
    u[1]=u[0]
    for n in tqdm(range(1,Nt-1)):
        for i in range(1,Nx-1): # All interior points
            u[n+1,i]=(1/(1+gamma*deltat))*((2*u[n, i]*(1-gamma*deltat))-(u[n-1,i]*(1-gamma*deltat))+(C**2)*(u[n,i+1]-2*u[n,i]+u[n,i-1]))
            
    sim.clearDirectory()
    for i in tqdm(range(Nt)):
        plt.plot(u[i])
        plt.xlim(0,Nx)
        plt.ylim(-5,5)
        plt.savefig('../..//Physics-Simulations/Images for simulation/graph'+str(i)+'.png') # dpi argument increases resolution
        plt.close('all')

In [48]:
dampedWaveSimulation(gaussianDistribution)

100%|████████████████████████████████████████████████████████████████████████| 1000/1000 [00:39<00:00, 25.10it/s]
